In [97]:
import os
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from pathlib import Path

In [98]:
CIRCLES_SOURCE_DIR = '../input/basicshapes/shapes/circles/'
SQUARES_SOURCE_DIR = '../input/basicshapes/shapes/squares/'
TRIANGLES_SOURCE_DIR = '../input/basicshapes/shapes/triangles/'

print(len(os.listdir(CIRCLES_SOURCE_DIR)))
print(len(os.listdir(SQUARES_SOURCE_DIR)))
print(len(os.listdir(TRIANGLES_SOURCE_DIR)))

In [99]:
train_circles_dir = "./shapes/training/circles"
Path(train_circles_dir).mkdir(parents=True, exist_ok=True)
train_squares_dir = "./shapes/training/squares"
Path(train_squares_dir).mkdir(parents=True, exist_ok=True)
train_triangles_dir = "./shapes/training/triangles"
Path(train_triangles_dir).mkdir(parents=True, exist_ok=True)

val_circles_dir = "./shapes/testing/circles"
Path(val_circles_dir).mkdir(parents=True, exist_ok=True)
val_squares_dir = "./shapes/testing/squares"
Path(val_squares_dir).mkdir(parents=True, exist_ok=True)
val_triangles_dir = "./shapes/testing/triangles"
Path(val_triangles_dir).mkdir(parents=True, exist_ok=True)

In [100]:
def split_data(source, training, testing, split_size):
    files = []
    
    for f in os.listdir(source):
        f_path = source + f
        if os.path.getsize(f_path):
            files.append(f)
   
    split_point = int(len(files) * split_size)
    random.shuffle(files)
    train_set = files[:split_point]
    val_set = files[split_point:]
    
    for f in train_set:
        copyfile(os.path.join(source, f), os.path.join(training, f))
        
    for f in val_set:
        copyfile(os.path.join(source, f), os.path.join(testing, f))

split_size = .7
split_data(CIRCLES_SOURCE_DIR, train_circles_dir, val_circles_dir, split_size)
split_data(SQUARES_SOURCE_DIR, train_squares_dir, val_squares_dir, split_size)
split_data(TRIANGLES_SOURCE_DIR, train_triangles_dir, val_triangles_dir, split_size)

In [101]:
print(len(os.listdir(train_circles_dir)))
print(len(os.listdir(train_squares_dir)))
print(len(os.listdir(train_triangles_dir)))
print(len(os.listdir(val_circles_dir)))
print(len(os.listdir(val_squares_dir)))
print(len(os.listdir(val_triangles_dir)))

In [102]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Parameters for our graph; we'll output images in a 3x4 configuration
nrows = 3
ncols = 4

pic_index = 0 # Index for iterating over images

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols, nrows)

pic_index+=4

next_cirlce = [os.path.join(train_circles_dir, fname) 
                for fname in os.listdir(train_circles_dir)[ pic_index-4:pic_index] 
               ]

next_square = [os.path.join(train_squares_dir, fname) 
                for fname in os.listdir(train_squares_dir)[ pic_index-4:pic_index]
               ]

next_triangle = [os.path.join(train_triangles_dir, fname) 
                for fname in os.listdir(train_triangles_dir)[ pic_index-4:pic_index]
               ]

for i, img_path in enumerate(next_cirlce+next_square+next_triangle):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [103]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip=True)

train_dir = "./shapes/training"
train_generator = train_datagen.flow_from_directory(train_dir,
                                    batch_size=30,
                                    class_mode='categorical',
                                    target_size=(28, 28))  

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_dir = "./shapes/testing"
validation_generator = validation_datagen.flow_from_directory(val_dir,
                                                batch_size=30,
                                                class_mode  = 'categorical',
                                                target_size = (28, 28))

In [106]:
class myCb(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get("val_accuracy") >= 0.9:
            print("Reached 90% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCb()

In [107]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2), 
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dense(3, activation='softmax')  
])

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator,
                    epochs=35,
                    steps_per_epoch=7,
                    validation_data = validation_generator,
                    validation_steps=3,
                    callbacks=[callbacks],
                    verbose = 1)

In [108]:
# Plot the chart for accuracy and loss on both training and validation

import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()